## Import Packages

This script makes use of the packages below.  
* 'sqlalchemy' is used to create a connection engine to our Postgres database.
* 'pandas' allows us to query our database using SQL while arraying our data calls into dataframes.
* 'numpy', 'seaborn', and 'matplotlib' are used for visualization purposes.
* 'plotly' used for visualization.



In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

plotly.__version__
plotly.offline.init_notebook_mode(connected=True)

### Import pre-processed data subset.

In [2]:
marketcheck = pd.read_pickle("listings.pkl", compression='infer')

print(marketcheck.head())

                   vin_ss vehicle_type_ss body_type_ss make_ss model_ss  \
231130  JHLRD78883C045742             SUV          SUV   HONDA     CR-V   
66452   JHLRE48777C090454             SUV          SUV   HONDA     CR-V   
144754  1HGEM225X1L115205             Car        Coupe   HONDA    CIVIC   
232386  2HGFG1B88BH518131             Car        Coupe   HONDA    CIVIC   
172683  2HGFA1F32AH537028             Car        Sedan   HONDA    CIVIC   

        year_is      fuel_type_ss engine_size_ss transmission_ss doors_is  \
231130     2003  Regular Unleaded            2.4       Automatic        4   
66452      2007  Regular Unleaded            2.4       Automatic        4   
144754     2001  Regular Unleaded            1.7       Automatic        2   
232386     2011  Regular Unleaded            1.8       Automatic        2   
172683     2010  Regular Unleaded            1.8       Automatic        4   

           ...      interior_color_ss_encoded exterior_color_ss_encoded  \
231130     

In [3]:
#Filters results from states not supported by  plotly map
filtered_df = marketcheck[(marketcheck['state_ss'] != 'GU') & (marketcheck['state_ss'] != 'PR') & (marketcheck['state_ss'] != 'WS')]

#Aggregate by state_ss, use reset_index() to be able to access group names from dataframework
aggregated_df = filtered_df[['state_ss','price_fs']].groupby(['state_ss'],as_index=False).agg({'price_fs':['mean','count']})
aggregated_df




state_ss      price_fs      
                     mean count
0        AL  17311.719136  1620
1        AR  18988.161232   552
2        AZ  17007.092142  1769
3        CA  17591.415311  7315
4        CO  15995.636364  1573
5        DE  14749.779221   154
6        FL  16989.165454  6872
7        GA  16561.787922  4239
8        HI  17751.879173   629
9        IA  16579.268266   917
10       ID  17570.631148   488
11       IL  18000.803080   909
12       IN  17480.053311  1782
13       KS  17654.353925   777
14       KY  16644.993997  1166
15       MD  17176.818058  1473
16       MI  17512.165658  1322
17       MN  17542.098069  1968
18       MO  17289.956879   974
19       MS  18057.668956   728
20       MT  17987.790614   277
21       NC  16947.700908  5617
22       ND  19096.563636   220
23       NE  17042.237519   661
24       NM  18512.606383   376
25       NV  17116.349925   663
26       NY  17391.759135  2682
27       OH  16041.287746  5345
28       OK  14855.512757  1215
29       OR  16847.895062  1134
30       PA  17177.888860  3815
31       SC  16762.577358  1855
32       SD  17213.671233   365
33       TN  16661.736768  2834
34       TX  17030.675448  4859
35       UT  17134.894576  1309
36       VA  15938.530102  1279
37       WA  16609.754826  1036
38       WI  17424.788690  1680
39       WV  19045.673684   285
40       WY  16880.132530    83

### Prepare Plotly Visualization

In [4]:
#Activate offline mode for splotly. If not done, visualization will not show since plotly works online by default.
plotly.offline.init_notebook_mode(connected=True)

#Define color scale

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


#Prepare text to show in states

def f(x):
    d = {}
    d['a_mean'] = x['a'].mean()
    d['a_count'] = x['a'].count()
    return pd.Series(d, index=['a_mean', 'a_count'])


aggregated_df['text'] = aggregated_df['state_ss']




In [5]:
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = aggregated_df['state_ss'],
        z = aggregated_df['price_fs']['count'].astype(float),
        locationmode = 'USA-states',
        text = aggregated_df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Listings")
        ) ]

layout = dict(
        title = 'HONDA Lisings by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
plotly.offline.iplot( fig, filename='d3-cloropleth-map' )